In [3]:
import os
import numpy as np
import pandas as pd

# =========================
# CONFIG
# =========================

DATA_TYPE = "ADP"  # change to "MSB" or "MBC" to compare

TRANSFORMATIONS_NOT_HUMAN = [
    "rawSN","pcaSN","degSN","degRandSN","degPCA_SN",
    "scviSN","scvi_LSshift_SN","degScviSN","degScviLSshift_SN"
]
TRANSFORMATIONS_HUMAN = [
    "rawSN","pcaSN","degSN","degIntSN","degRandSN","degOtherSN",
    "degPCA_SN","scviSN","scvi_LSshift_SN","degScviSN","degScviLSshift_SN"
]

IS_HUMAN = (DATA_TYPE != "MSB")
TRANSFORMATIONS = TRANSFORMATIONS_HUMAN if IS_HUMAN else TRANSFORMATIONS_NOT_HUMAN

BASE_REFS = ["sc_raw", "sn_raw", "degIntAllSN"]

ROOT = os.path.abspath(os.path.join(os.getcwd(), "scripts"))
IMPORT_PATH = os.path.abspath(os.path.join(ROOT, "../../", "data", "deconvolution", DATA_TYPE))
MIXTURE_FILE = os.path.join(IMPORT_PATH, "pseudobulks.csv")

print("IMPORT_PATH:", IMPORT_PATH)
print("MIXTURE_FILE:", MIXTURE_FILE)

def load_mixture(path):
    mix = pd.read_csv(path, index_col=0)
    print("Loaded mixture_data:", mix.shape)
    return mix

def basic_value_stats_sample(mat: pd.DataFrame, max_sample=300_000):
    vals = mat.to_numpy().astype(float).ravel()
    vals = vals[np.isfinite(vals)]
    if vals.size == 0:
        return dict(min=np.nan, max=np.nan, mean=np.nan,
                    frac_lt1=np.nan, frac_nonint=np.nan,
                    any_negative=False, any_nan=False, any_inf=False)
    if vals.size > max_sample:
        idx = np.random.choice(vals.size, size=max_sample, replace=False)
        vals = vals[idx]
    stats = {
        "min": float(vals.min()),
        "max": float(vals.max()),
        "mean": float(vals.mean()),
        "frac_lt1": float((vals < 1).mean()),
        "frac_nonint": float((vals != np.round(vals)).mean()),
        "any_negative": bool((vals < 0).any()),
        "any_nan": bool(np.isnan(vals).any()),
        "any_inf": bool(np.isinf(vals).any()),
    }
    return stats

def looks_like_log_or_heavy_norm(stats, log_max=20, high_frac_lt1=0.8):
    if np.isnan(stats["max"]):
        return True
    if stats["any_negative"] or stats["any_nan"] or stats["any_inf"]:
        return True
    if stats["max"] < log_max and stats["frac_lt1"] > high_frac_lt1:
        return True
    return False

def check_reference(mixture_df, signal_path, cell_state_path, ref_name):
    info = {
        "ref": ref_name,
        "signal_exists": os.path.exists(signal_path),
        "cell_state_exists": os.path.exists(cell_state_path),
        "ok": True,
        "reason": [],
        "n_genes_overlap": 0,
        "n_cells": np.nan,
        "stats_min": np.nan,
        "stats_max": np.nan,
        "stats_mean": np.nan,
        "stats_frac_lt1": np.nan,
        "stats_frac_nonint": np.nan,
        "looks_logish": np.nan,
        "labels_match_cells": np.nan,
        "transform": None,
    }

    if not info["signal_exists"] or not info["cell_state_exists"]:
        info["ok"] = False
        info["reason"].append("missing signal or cell_state file")
        return info

    try:
        signal_df = pd.read_csv(signal_path, index_col=0)
    except Exception as e:
        info["ok"] = False
        info["reason"].append(f"failed to load signal: {e}")
        return info

    try:
        anno_df = pd.read_csv(cell_state_path)
    except Exception as e:
        info["ok"] = False
        info["reason"].append(f"failed to load cell_state: {e}")
        return info

    # Gene intersection
    common_genes = mixture_df.index.intersection(signal_df.index)
    info["n_genes_overlap"] = len(common_genes)
    if len(common_genes) < 500:
        info["ok"] = False
        info["reason"].append(f"<500 overlapping genes ({len(common_genes)})")

    if len(common_genes) == 0:
        info["ok"] = False
        info["reason"].append("no overlapping genes")
        return info

    signal_sub = signal_df.loc[common_genes]
    info["n_cells"] = signal_sub.shape[1]

    # Value stats
    stats = basic_value_stats_sample(signal_sub)
    info["stats_min"] = stats["min"]
    info["stats_max"] = stats["max"]
    info["stats_mean"] = stats["mean"]
    info["stats_frac_lt1"] = stats["frac_lt1"]
    info["stats_frac_nonint"] = stats["frac_nonint"]
    info["looks_logish"] = looks_like_log_or_heavy_norm(stats)

    if stats["any_negative"]:
        info["ok"] = False
        info["reason"].append("negative values in signal")
    if stats["any_nan"] or stats["any_inf"]:
        info["ok"] = False
        info["reason"].append("NaN/Inf values in signal")
    if info["looks_logish"]:
        info["reason"].append("values look log-ish / heavily normalized")

    # Annotation
    if "cell_type" not in anno_df.columns:
        info["ok"] = False
        info["reason"].append("cell_state has no 'cell_type' column")
        return info

    labels = anno_df["cell_type"].astype(str)
    if len(labels) != signal_sub.shape[1]:
        info["ok"] = False
        info["reason"].append(f"len(cell_type)={len(labels)} != n_cells={signal_sub.shape[1]}")
        info["labels_match_cells"] = False
    else:
        info["labels_match_cells"] = True

    return info

# =========================
# Run checks
# =========================

mixture_df = load_mixture(MIXTURE_FILE)
results = []

# Base refs
for base_ref in BASE_REFS:
    sig_file = os.path.join(IMPORT_PATH, f"{base_ref}_signal.csv")
    anno_file = os.path.join(IMPORT_PATH, f"{base_ref}_cell_state.csv")
    info = check_reference(mixture_df, sig_file, anno_file, ref_name=base_ref)
    info["transform"] = "base"
    results.append(info)

# Cell types from sc_raw_cell_state.csv
sc_raw_cell_state_file = os.path.join(IMPORT_PATH, "sc_raw_cell_state.csv")
if os.path.exists(sc_raw_cell_state_file):
    sc_raw_cell_state = pd.read_csv(sc_raw_cell_state_file)
    unique_cell_types = sorted(sc_raw_cell_state["cell_type"].unique())
else:
    unique_cell_types = []

# All transforms
for ct in unique_cell_types:
    ct_clean = str(ct).replace(" ", "_")
    for trans in TRANSFORMATIONS:
        ref_prefix = f"ref_{ct_clean}_{trans}"
        sig_file = os.path.join(IMPORT_PATH, f"{ref_prefix}_signal.csv")
        anno_file = os.path.join(IMPORT_PATH, f"{ref_prefix}_cell_state.csv")
        info = check_reference(mixture_df, sig_file, anno_file, ref_name=ref_prefix)
        info["transform"] = trans
        results.append(info)

results_df = pd.DataFrame(results)

results_df["ok_flag"] = results_df["ok"].astype(int)
results_df = results_df.sort_values(
    by=["transform", "ok_flag", "n_genes_overlap", "n_cells"],
    ascending=[True, False, False, False]
)

pd.set_option("display.max_rows", 300)
pd.set_option("display.max_colwidth", 200)

print("\n===== SUMMARY OF ALL TRANSFORMS FOR DATA_TYPE =", DATA_TYPE, "=====\n")
display_cols = [
    "ref", "transform", "ok", "n_genes_overlap", "n_cells",
    "stats_min", "stats_max", "stats_mean",
    "stats_frac_lt1", "stats_frac_nonint",
    "looks_logish", "labels_match_cells", "reason"
]
display(results_df[display_cols])


IMPORT_PATH: /projects/aivich@xsede.org/deconvolution_sc_sn_comparison/data/deconvolution/ADP
MIXTURE_FILE: /projects/aivich@xsede.org/deconvolution_sc_sn_comparison/data/deconvolution/ADP/pseudobulks.csv
Loaded mixture_data: (28129, 1000)

===== SUMMARY OF ALL TRANSFORMS FOR DATA_TYPE = ADP =====



,ref,transform,ok,n_genes_overlap,n_cells,stats_min,stats_max,stats_mean,stats_frac_lt1,stats_frac_nonint,looks_logish,labels_match_cells,reason
0,sc_raw,base,True,28129,2155,0.0,279.00000,0.123373,0.940687,0.000000,False,True,[]
1,sn_raw,base,True,28129,2155,0.0,356.00000,0.073470,0.953487,0.000000,False,True,[]
2,degIntAllSN,base,True,26350,2155,0.0,710.00000,0.068073,0.960973,0.000000,False,True,[]
61,ref_Preadipocytes_degIntSN,degIntSN,True,26350,4083,0.0,409.00000,0.070077,0.959623,0.000000,False,True,[]
39,ref_Macrophages_degIntSN,degIntSN,True,26350,3736,0.0,338.00000,0.075787,0.958753,0.000000,False,True,[]
17,ref_Endothelial_Cells_degIntSN,degIntSN,True,26350,3402,0.0,372.00000,0.080927,0.958223,0.000000,False,True,[]
6,ref_Dendritic_Cells_degIntSN,degIntSN,True,26350,2155,0.0,410.00000,0.093717,0.952050,0.000000,False,True,[]
28,ref_Immature_NK_T_Cells_degIntSN,degIntSN,True,26350,2155,0.0,559.00000,0.094100,0.952307,0.000000,False,True,[]
50,ref_Monocytes_degIntSN,degIntSN,True,26350,2155,0.0,341.00000,0.095697,0.951997,0.000000,False,True,[]
72,ref_T_Cells_degIntSN,degIntSN,True,26350,2155,0.0,787.00000,0.095050,0.952030,0.000000,False,True,[]


In [1]:
import os, glob, re

# === CONFIG: adjust if needed ===
BASE_DIR = "/projects/aivich@xsede.org/deconvolution_sc_sn_comparison"
dataset = "Real_ADP"
method = "SCDC"
mode = "perdonor"  # this is the mode that failed

# --- paths ---
DATA_PATH = os.path.join(BASE_DIR, "data", "deconvolution", dataset)
RESULTS_PATH = os.path.join(BASE_DIR, "results", dataset)

print("DATA_PATH   :", DATA_PATH)
print("RESULTS_PATH:", RESULTS_PATH)

# --- load donor_list from donors.csv (same logic as process_results.py) ---
donors_csv = os.path.join(DATA_PATH, "donors.csv")
if not os.path.exists(donors_csv):
    raise FileNotFoundError(f"donors.csv not found at {donors_csv}")

with open(donors_csv) as fh:
    donor_list = [d.strip() for d in fh.readline().split(",") if d.strip()]

print("\nDonor list loaded (first few):", donor_list[:5])
print("Total donors:", len(donor_list))

# --- valid transforms (same as your script) ---
valid_transforms = {
    "rawSN",
    "pcaSN",
    "degSN",
    "degRandSN",
    "degIntSN",
    "degOtherSN",
    "degPCA_SN",
    "degpcaSN",
    "scviSN",
    "scvi_LSshift_SN",
    "degScviSN",
    "degScviLSshift_SN",
    "degIntAllSN",
}

# --- this is your CURRENT parse_bp_filename (buggy version) ---
def parse_bp_filename_buggy(path: str, mode: str, method: str):
    fname = os.path.basename(path)
    core = re.sub(fr"_{method}_proportions\.csv$", "", fname)

    # simulation branch (not used here)
    if mode == "simulation":
        if core in {"sc_raw", "sn_raw"}:
            return None, core
        if core.startswith("ref_"):
            tail = core[4:]
            for t in valid_transforms:
                if tail.endswith("_" + t):
                    return None, t
        return None, None

    # real bulk branch (not used here)
    if mode == "real":
        if core in {"sc_raw_real", "sn_raw_real"}:
            return None, core
        if core.startswith(("sc_raw_", "sn_raw_")):
            parts = core.split("_")
            transform = "_".join(parts[:2])
            return None, transform
        if core.startswith("ref_real_"):
            trans = core[9:]
            if trans in valid_transforms:
                return None, trans
            if trans.lower() in valid_transforms:
                return None, trans.lower()
        return None, None

    # ---------- perdonor branch (this is what we're testing) ----------
    if mode == "perdonor" and core.startswith("ref_"):
        tail = core[4:]  # e.g. "Hs_SAT_01-1_degSN"
        if "_" not in tail:
            return None, None
        donor, transform = tail.split("_", 1)  # <-- THIS is the suspicious line

        print(f"DEBUG parse: fname={fname}")
        print(f"       core={core}")
        print(f"       tail={tail}")
        print(f"       donor={donor!r}, transform={transform!r}")
        print(f"       donor in donor_list? {donor in donor_list}\n")

        if donor not in donor_list:
            return None, None
        if transform in valid_transforms or transform.lower() in valid_transforms:
            transform = transform if transform in valid_transforms else transform.lower()
            return donor, transform
        return None, None

    return None, None


# --- collect files and test parsing ---
bp_files = sorted(glob.glob(os.path.join(RESULTS_PATH, f"*_{method}_proportions.csv")))
print(f"\nFound {len(bp_files)} '{method}' files.\n")

accepted = []
skipped = []

for fp in bp_files:
    donor, trans = parse_bp_filename_buggy(fp, mode=mode, method=method)
    fname = os.path.basename(fp)
    if trans is None:
        skipped.append(fname)
    else:
        accepted.append((fname, donor, trans))

print("\n=== SUMMARY ===")
print("Accepted files (parsed successfully):", len(accepted))
for fname, donor, trans in accepted[:10]:
    print(f"  OK   {fname}  -> donor={donor}, transform={trans}")

print("\nSkipped files (unrecognised):", len(skipped))
for fname in skipped[:10]:
    print(f"  SKIP {fname}")

DATA_PATH   : /projects/aivich@xsede.org/deconvolution_sc_sn_comparison/data/deconvolution/Real_ADP
RESULTS_PATH: /projects/aivich@xsede.org/deconvolution_sc_sn_comparison/results/Real_ADP

Donor list loaded (first few): ['Hs_SAT_12-1', 'Hs_SAT_13-1', 'Hs_SAT_266-1', 'Hs_SAT_01-1', 'Hs_SAT_02-1']
Total donors: 12

Found 158 'SCDC' files.

DEBUG parse: fname=ref_Hs_SAT_01-1_degIntAllSN_SCDC_proportions.csv
       core=ref_Hs_SAT_01-1_degIntAllSN
       tail=Hs_SAT_01-1_degIntAllSN
       donor='Hs', transform='SAT_01-1_degIntAllSN'
       donor in donor_list? False

DEBUG parse: fname=ref_Hs_SAT_01-1_degIntSN_SCDC_proportions.csv
       core=ref_Hs_SAT_01-1_degIntSN
       tail=Hs_SAT_01-1_degIntSN
       donor='Hs', transform='SAT_01-1_degIntSN'
       donor in donor_list? False

DEBUG parse: fname=ref_Hs_SAT_01-1_degOtherSN_SCDC_proportions.csv
       core=ref_Hs_SAT_01-1_degOtherSN
       tail=Hs_SAT_01-1_degOtherSN
       donor='Hs', transform='SAT_01-1_degOtherSN'
       donor in d